In [1]:
import pandas as pd
import sqlalchemy
import numpy as np
import seaborn as sns
import os
import matplotlib.pyplot as plt
from sqlalchemy.exc import SQLAlchemyError
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from db_queries import username, password, dsn, dbhostname, service_name, dbtables, querys
from table_functions import *
from analyze_visualisation import *

In [2]:
data = {}

try:
    sqlalchemy_engine="oracle+cx_oracle://"+username+":"+password+"@"+dbhostname+"/?service_name="+service_name
    engine = sqlalchemy.create_engine(sqlalchemy_engine, arraysize=1000)
    for table, query in zip(dbtables, querys):
        data.update({table: pd.read_sql(query, engine)})
except SQLAlchemyError as e:
    print(e)

data = drop_unused_columns(data)

In [ ]:
final_table = combine_final_table(data)
final_table = create_final_status(final_table)

In [4]:
save_df_to_csv(final_table, 'final_table.csv')

In [2]:
final_table = read_csv('final_table.csv')

In [3]:
final_table = standarize_data(final_table)
final_table = normalize_data(final_table)

In [4]:
x_train, x_valid, x_test, y_train, y_valid, y_test = split_data(final_table)

In [ ]:
#describe_our_data(final_table)

In [ ]:
analyze_data(final_table)

In [4]:
# do modelowania:
'czas_fazy_1', 'czas_fazy_2', 'czas_fazy_3', 'max_predkosc', 'cisnienie_tloka', 'cisnienie_koncowe', 'nachdruck_hub', 'anguss', 'oni_temp_curr_f1', 
'oni_temp_curr_f2', 'oni_temp_fore_f1', 'oni_temp_fore_f2', 'vds_air_pressure', 'vds_vac_hose1', 'vds_vac_hose2', 'vds_vac_tank', 'vds_vac_valve1', 'vds_vac_valve2', 'czas_taktu',
#assigment_1-28, flow_1-28, set_point_1-28, start_delay_1-28. temp_1-28, working_mode_1-28

# nie do modelowania
'nr_dgm', 'rodzaj_kontroli', 'id_dmc_DMC', 'kod_pola', 'rodzaj_uszkodzenia', 'temp_workpiece', 'temp_hydraulics', 'pressure_pcf_1', 'pressure_pcf_2', 'pressure_pcf_3', 
'cisnienie', 'przeciek', 'temperaturatestu', 'temp_pieca'

#nasz parametr klasy:
'our_final_status'

columns = ['nr_dgm', 'rodzaj_kontroli', 'id_dmc_DMC', 'kod_pola', 'rodzaj_uszkodzenia', 'temp_workpiece', 'temp_hydraulics', 'pressure_pcf_1', 'pressure_pcf_2', 'pressure_pcf_3', 
'cisnienie', 'przeciek', 'temperaturatestu', 'temp_pieca']
final_table.drop(columns=columns, inplace=True)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(final_table, target, train_size=0.80, random_state=42, stratify=target)

print(f'data_train: {x_train.shape}')
print(f'data_test: {x_test.shape}')
print(f'target_train: {y_train.shape}')
print(f'target_test: {y_test.shape}')
print(f'target_train\n {y_train.value_counts()}')
print(f'target_test\n {y_test.value_counts()}')

In [ ]:
x_valid, x_test, y_valid, y_test = train_test_split(x_test, y_test, train_size=0.5, random_state=42, stratify=y_test)

print(f'data_train: {x_valid.shape}')
print(f'data_test: {x_test.shape}')
print(f'target_train: {y_valid.shape}')
print(f'target_test: {y_test.shape}')
print(f'target_train\n {y_valid.value_counts()}')
print(f'target_test\n {y_test.value_counts()}')